In [25]:
import os
import json
from sklearn.metrics import mean_absolute_error, median_absolute_error
import scipy.stats as st
import numpy as np

In [3]:
os.chdir("..")

## Importing the metadata

In [4]:
METADATA_PATH = "data/silver/metadata"

metadata = {}
for metadata_filename in os.listdir(METADATA_PATH):
    with open(f"{METADATA_PATH}/{metadata_filename}") as metadata_file:
        dish_metadata = json.load(metadata_file)
        metadata[dish_metadata["dish_id"]] = dish_metadata

## Importing the results

In [5]:
RESULTS_PATH = "results"

models_names = [
    "gemini",
    "gemini_split"
]

models_results = {}

for model_name in models_names:
    model_results_path = f"{RESULTS_PATH}/{model_name}"
    
    results = {}
    for result_filename in os.listdir(model_results_path):
        with open(f"{model_results_path}/{result_filename}") as result_file:
            result = json.load(result_file)
            results[result["dishId"]] = result
    
    models_results[model_name] = results

## Evaluating results

In [6]:
def confidence_interval(data, confidence=0.9, decimals=1):
    mean = np.mean(data)
    standard_error = st.sem(data, nan_policy="omit")
    
    z_score = st.norm.ppf((1 + confidence) / 2)
    margin_of_error = standard_error * z_score

    lower_bound = round(mean - margin_of_error, decimals)
    upper_bound = round(mean + margin_of_error, decimals)

    return lower_bound, upper_bound


In [7]:
CONFIDENCE_LEVELS = [0.85, 0.9, 0.95, 0.99]

In [ ]:
for model_name, model_results in models_results.items():
    Y_test_carb = []
    Y_pred_carb = []

    Y_test_calories = []
    Y_pred_calories = []

    processing_times = []

    for dish_id, dish_metadata in metadata.items():
        if dish_id in model_results:
            result = model_results.get(dish_id)

            Y_test_carb.append(dish_metadata["total_carb"])
            Y_pred_carb.append(result["totalCarbohydrates"])

            Y_test_calories.append(dish_metadata["total_calories"])
            Y_pred_calories.append(result["totalCalories"])

            processing_times.append(result["elapsedTime"])

    mae_carb = mean_absolute_error(Y_test_carb, Y_pred_carb)
    median_carb = median_absolute_error(Y_test_carb, Y_pred_carb)
    residuals_carb = np.abs(np.array(Y_pred_carb) - np.array(Y_test_carb))
    sem_carb = st.sem(residuals_carb)
    ci_carb = st.mstats.median_cihs(residuals_carb, alpha=0.05)
    
    mae_calories = mean_absolute_error(Y_test_calories, Y_pred_calories)
    median_calories = median_absolute_error(Y_test_calories, Y_pred_calories)
    residuals_calories = np.abs(np.array(Y_pred_calories) - np.array(Y_test_calories))
    sem_calories = st.sem(residuals_calories)
    ci_calories = st.mstats.median_cihs(residuals_calories, alpha=0.05)

    avg_time = np.mean(processing_times)
    median_time = np.median(processing_times)

    print(f"{model_name.capitalize()}")
    print(f"-> Median carbohydrates error = {median_carb:.1f} ({ci_carb[0]:.1f} - {ci_carb[1]:.1f}) g")
    print(f"-> MAE carbohydrates = {mae_carb:.1f} g (SEM = {sem_carb:.1f} g)")
    
    print(f"-> Median calories error = {median_calories:.1f} ({ci_calories[0]:.1f} - {ci_calories[1]:.1f}) kcal")
    print(f"-> MAE calories = {mae_calories:.1f} kcal (SEM = {sem_calories:.1f} kcal)")
    print(f"-> Average processing time = {avg_time:.2f} s")
    print(f"-> Median processing time = {median_time:.2f} s")
    print()

Gemini
-> Median carbohydrates error = 10.3 (8.9 - 12.3) g
-> MAE carbohydrates = 17.2 g (SEM = 0.8 g)
-> Median calories error = 75.1 (65.0 - 83.4) kcal
-> MAE calories = 113.6 kcal (SEM = 5.3 kcal)
-> Average processing time = 2.76 s
-> Median processing time = 2.41 s

Gemini_split
-> Median carbohydrates error = 7.2 (6.4 - 8.5) g
-> MAE carbohydrates = 14.9 g (SEM = 1.2 g)
-> Median calories error = 63.4 (55.6 - 73.1) kcal
-> MAE calories = 113.5 kcal (SEM = 6.6 kcal)
-> Average processing time = 3.74 s
-> Median processing time = 2.99 s

